# **Informe técnico**

## **Locality Sensitive Hashing**
Locality-Sensitive Hashing (LSH) es un método de búsqueda de vecinos más cercanos, agrupa elementos similares en los mismos buckets mediante una función hash especialmente diseñada.
A diferencia de los esquemas de hashing tradicionales, que buscan minimizar colisiones para distribuir los datos uniformemente, LSH busca maximizar la probabilidad de colisión entre elementos similares.

En lugar de buscar entre todos los elementos, hace lo siguiente:

*   Aplica una función hash sensible a similitud
*   Agrupa los vectores en buckets
*   Busca solo dentro del bucket correspondiente y vecinos si hace falta

Es útil cuando se tienen datasets enormes y se quiere reducir el espacio de búsqueda sin comparar cada punto con todos los demás.

LSH emplea una función hash sensible a similitud, de manera que si dos puntos están cerca en el espacio original, tienen una alta probabilidad de caer en el mismo bucket después de aplicar LSH."

### Proyecciones aleatorias con hiperplanos
Un hiperplano divide el espacio en dos mitades. Al proyectar un punto sobre varios hiperplanos, clasificamos el punto según el lado en el que cae respecto a cada uno.

Esto permite dividir el espacio en regiones poligonales donde los puntos cercanos tienen una mayor probabilidad de terminar en el mismo bucket.

### Diseño del algoritmo

Para implementar este algoritmo, se definió una clase LSH con los métodos train(num_vector), query(query_vec, k, max_search_radius) y search_nearby_bins(query_bin_bits, table, search_radius, initial_candidates). Esta clase permite indexar los datos en buckets usando proyecciones aleatorias, y luego buscar vecinos aproximados mediante hashing. Como el algoritmo trabaja con múltiples vectores, se generan N códigos binarios y se agrupan en una tabla hash.



In [ ]:
from copy import copy
from itertools import combinations
import numpy as np
from pandas import DataFrame
from sklearn.metrics.pairwise import pairwise_distances


class LSH:
    def __init__(self, data):
        # La data se almacena en un vector mxn, donde m es la cantidad de datos y n la longitud del vector que las representa
        self.data = data
        self.model = None

### generate_random_vectors(num_vector, dim)
Generamos `k` vectores aleatorios:  
   $$
   a_1, a_2, \dots, a_k \in \mathbb{R}^d
   $$
   que actuarán como hiperplanos que dividen el espacio.

In [ ]:
def __generate_random_vectors(self, num_vector, dim):
        vectors = np.random.randn(dim, num_vector)
        return vectors

### train(num_vector, seed)
El método train construye la tabla hash de LSH proyectando cada vector sobre hiperplanos aleatorios. A partir de esas proyecciones, asigna a cada vector un código binario que determina su bucket, esta etapa indexa eficientemente los datos antes de realizar búsquedas.

* Para cada vector de datos $x \in \mathbb{R}^d$, calculamos su proyección sobre cada hiperplano:
   $$
   h_i(x) =
   \begin{cases}
   1 & \text{si } x \cdot a_i \geq 0 \\
   0 & \text{si } x \cdot a_i < 0
   \end{cases}
   $$

  Estos vectores se generan usualmente con distribución gaussiana estándar
  $$
  a_i \sim \mathcal{N}(0, 1)^d
  $$

  Esto asegura que los hiperplanos estén uniformemente distribuidos sobre la esfera d-dimensional, sin una dirección preferente. Es decir, cada dirección es igualmente probable, lo cual es ideal para no introducir sesgo espacial.

* Esto genera un vector binario de `k` bits, donde cada bit indica en qué lado del hiperplano cayó el punto.

* Ese vector binario se interpreta como un entero:
   - Cada combinación de bits representa un bucket
   - Todos los puntos que caen en el mismo bucket son considerados potenciales vecinos

In [ ]:

    def train(self, num_vector, seed=None):
        # Obtener la longitud n de los vectores
        dim = self.data.shape[1]
        if seed is not None:
            np.random.seed(seed)

        random_vectors = self.__generate_random_vectors(num_vector, dim)
        powers_of_two = 1 << np.arange(num_vector - 1, -1, -1)

        table = {}

        # Dividir los puntos de datos en buckets
        bin_index_bits = (self.data.dot(random_vectors) >= 0)

        # Codificar los bits del índice de bucket como enteros
        bin_indices = bin_index_bits.dot(powers_of_two)

        # Actualizar la tabla para que table[i] sea la lista de documentos con índice igual a i
        for data_index, bin_index in enumerate(bin_indices):
            if bin_index not in table:
                # Si aún no existe una lista para este bucket, se inicializa vacía
                table[bin_index] = []
            # Se añade el índice del documento al bucket correspondiente
            table[bin_index].append(data_index)

        self.model = {
            'bin_indices': bin_indices,
            'table': table,
            'random_vectors': random_vectors,
            'num_vector': num_vector
        }
        return self


**Complejidad**: Cada uno de los $N$ vectores de dimensión $d$ se proyecta sobre $k$ hiperplanos.  Esta operación se realiza mediante un producto matricial $N \times d \cdot d \times k$.
$$
  \mathcal{O}(N \cdot d \cdot k)
  $$



## search_nearby_bins(): Búsqueda en el bucket original y buckets vecinos



*   Recupera todos los elementos que están en el mismo bucket que la consulta, esto es rápido, pero puede no ser suficiente si el bucket está vacío o tiene pocos elementos.
*   Explora buckets con códigos similares al de la consulta, esto se hace mediante un parámetro llamado radio de búsqueda `r`, que indica cuántos bits del código binario se pueden invertir para generar buckets "cercanos".
 - Radio bajo: Busca solo en el bucket original o en los más cercanos. Es rápido, pero puede omitir vecinos relevantes si los datos están dispersos.
 - Radio alto: Explora más buckets y recupera más vecinos posibles. Aumenta la precisión, pero también el costo computacional.

In [ ]:
def __search_nearby_bins(self, query_bin_bits, table, search_radius=2, initial_candidates=set()):
        num_vector = self.model['num_vector']
        powers_of_two = 1 << np.arange(num_vector - 1, -1, -1)

        candidate_set = copy(initial_candidates)

        for different_bits in combinations(range(num_vector), search_radius):
            alternate_bits = copy(query_bin_bits)
            for i in different_bits:
                alternate_bits[i] = 1 if alternate_bits[i] == 0 else 0

            # Convertir el nuevo vector binario en un índice entero
            nearby_bin = alternate_bits.dot(powers_of_two)

            # Obtener la lista de elementos que pertenecen a este bucket cercano
            if nearby_bin in table:
                candidate_set.update(table[nearby_bin])

        return candidate_set

**Complejidad**: Se proyecta el query, se buscan buckets vecinos y se calcula la distancia real a cada candidato recuperado.

$$
  \mathcal{O}(R \cdot B + C \cdot d)
  $$

Donde:
  - $R$ = número de buckets explorados según el radio
  - $B$ = número promedio de puntos por bucket
  - $C$ = número total de candidatos recolectados
  - $d$ = dimensión de cada vector



### query(query_vec, k, max_search_radius, initial_candidates)
El método query permite recuperar los vecinos más cercanos a un vector de consulta, explorando buckets en orden creciente de radio. Utiliza las mismas proyecciones que en el entrenamiento para ubicar la consulta en el espacio hash.

* Se proyecta el vector de consulta sobre los mismos vectores aleatorios usados en el entrenamiento.
* Se obtiene el código binario del query y se convierte a un entero, esto define el bucket inicial.
* Luego, se realiza la exploración por radio:
   - Se buscan candidatos dentro del bucket original (`r = 0`)
   - Luego, se exploran buckets vecinos a medida que aumenta el radio, hasta `max_search_radius`.
   - Los buckets vecinos se generan invirtiendo combinaciones de bits del código binario
* Todos los puntos encontrados se recopilan en un conjunto de candidatos únicos.
* Se calcula la distancia euclidiana real entre el query y cada candidato.
* Finalmente, *se* devuelven los `k` puntos más cercanos ordenados por distancia.


In [ ]:
def query(self, query_vec, k, max_search_radius, initial_candidates=set()):
        if not self.model:
            print('Modelo aún no entrenado')
            exit(-1)

        data = self.data
        table = self.model['table']
        random_vectors = self.model['random_vectors']

        bin_index_bits = (query_vec.dot(random_vectors) >= 0).flatten()

        candidate_set = set(initial_candidates)
        # Buscar en buckets vecinos y recolectar candidatos
        for search_radius in range(max_search_radius + 1):
            candidate_set.update( self.__search_nearby_bins(
                    bin_index_bits,
                    table,
                    search_radius,
                    candidate_set
                )
            )

        # Ordenar candidatos por su distancia real al vector de consulta
        if not list(candidate_set):
            return DataFrame(columns=["id", "distance"])
        candidates = data[np.array(list(candidate_set)), :]
        nearest_neighbors = DataFrame({'id': list(candidate_set)})
        nearest_neighbors['distance'] = pairwise_distances(candidates, query_vec.reshape(1, -1), metric='euclidean').flatten()

        return nearest_neighbors.nsmallest(k, 'distance')

**Complejidad:**  Esta función genera todas las combinaciones posibles de bits cambiados hasta un radio $R$, lo que puede crecer rápidamente cuando $k$ o $R$ aumentan.
  $$
  \mathcal{O}\left( \sum_{r=0}^{R} \binom{k}{r} \right)
  $$

Aunque el peor caso puede ser costoso, LSH es eficiente en la práctica porque se limita a pocos bits, es decir $k$ pequeño, y radios bajos como $R = 0$ o $1$ suelen ser suficientes

## **Product Quantization**

El proceso de cuantificación se basa en comprimir datos para que ocupen menos espacio mediante la reducción del universo de valores de los datos. Es decir, se mapean valores continuos a valores discretos. En un contexto vectorial, un cuantificador se puede definir como una función $q$ que mapea un vector $x$ de $D$ dimensiones a un vector $q(x) \in \mathcal{C} = \{c_i;i \in \mathcal{I}\}$, siendo $\mathcal{I}$ el conjunto de índices finito $\{0,...,k-1\}$. Los valores $c_i$ son llamados centroides y el conjunto de centroides $\mathcal{C}$ es llamado *codebook*.

\begin{align}
\underbrace{x_1, \ldots, x_{D^*}}_{u_1(x)}, \ldots, \underbrace{x_{D - D^* + 1}, \ldots, x_D}_{u_m(x)}
\rightarrow q_1(u_1(x)), \ldots, q_m(u_m(x))
\end{align}

En este caso, el vector $x$ es dividido en $m$ subvectores de dimensión $D^* = D/m$, para los cuales se usan $m$ cuantificadores distintos Cada cuantificador $q_j$ tiene asociado un conjunto de índices $\mathcal{I}_j$, un *codebook* $\mathcal{C}_j$ y centroides $c_{j,i}$.

Un valor de reproducción de un PQ es un valor que permite recuperar el conjunto de centroides y (aproximadamente) el vector original. Este se identifica como el producto de índices $\mathcal{I}=\mathcal{I_1}\times\mathcal{I_2}\ldots\times\mathcal{I_m}$, cuyo *codebook* asociado sería $\mathcal{C}=\mathcal{C_1}\times\mathcal{C_2}\ldots\times\mathcal{C_m}$. En la práctica, el proceso de PQ resulta en representar un vector como un conjunto de índices en $\mathcal{I}$ en lugar de los centroides, reduciendo considerablemente la memoria necesaria para almacenar los vectores. Si cada subvector tiene $k^*$ centroides generados, el total de centroides de $C$ es de $k=(k^*)^m$.

### Diseño del algoritmo

Para implementar este algoritmo, se definió una clase `ProductQuantizer` con los métodos `fit(X)`, `encode(X)`, `decode(codes)` y `approximate_knn(query, codes)`. Se debe tomar en cuenta que no se suele trabajar con un solo vector, sino con $N$ vectores, por lo que los conceptos antes expuestos deben ser tomadas para $N$ datos. Es decir, $N$ *codebooks* y $N$ conjuntos de índices.

In [ ]:
class ProductQuantizer:
    def __init__(self, M=4, Ks=256):
        self.M = M
        self.Ks = Ks
        self.codebooks = []

#### fit(X)

En este método, se genera el *codebook* como atributo de la clase a partir de un conjunto de vectores. Se apoya de la librería `scikit-learn` para generar los centroides a partir del algoritmo *k-means* de clustering. Cuando se tiene un conjunto de vectores N, se obtienen los centroides a partir de los N subvectores de dimensión D/m.

Se optó usar un algoritmo ya implementado de *k-means* para centrarnos solo en el algortimo de PQ. Se optó en particular por la librería `scikit-learn` por su facilidad de uso y su implementación directa para usarlo en el algoritmo.


In [ ]:
    def fit(self, X):
        N, D = X.shape
        if not D % self.M == 0:
            raise ValueError(f"Dimensión {D} debe ser divisible por {self.M}.")
        self.Ds = D // self.M  # dimensión de cada subvector

        # Calcular k-means
        self.codebooks = []
        for m in range(self.M):
            # Extraer sub-vectores
            sub_vectors = X[:, m*self.Ds:(m+1)*self.Ds]
            kmeans = KMeans(n_clusters=self.Ks, random_state=42, n_init=10)
            kmeans.fit(sub_vectors)
            self.codebooks.append(kmeans.cluster_centers_)


##### Complejidad

- $N$: Número de vectores
- $D$: Dimensionalidad de subvectores
- $M$: Número de subvectores
- $N$: Número de centroides por *codebook*.

Se realizan k-means sobre $N$ vectores de dimensión $D/M$ para hallar $K$ centroides un total de $M$ veces. Asumiendo que la operación de k-means es constante, la complejidad asciende a:
\begin{align}
\mathcal{O}(NKD)
\end{align}

#### encode(X)

En este método se codifica un vector $X$ con el codebook ya entrenado. Para esto, se calcula la distancia de los $M$ subvectores de $X$ con todos los centroides generados por k-means para cada subespacio. Luego, almaceno los índices de los centroides en el *codebook* para los más cercanos.

In [ ]:
    def encode(self, X):
        if self.codebooks is []:
            print("! No se tiene codebook definido \n!",
                  "Llama a fit con el conjunto de vectores para generar codebook")
            return
        N, D = X.shape
        codes = np.empty((N, self.M), dtype=np.uint8)
        for m in range(self.M):
            sub_vectors = X[:, m*self.Ds:(m+1)*self.Ds]
            centroids = self.codebooks[m]
            # Distancia a centroides
            distances = np.linalg.norm(sub_vectors[:, None, :] - centroids[None, :, :], axis=2)
            # Almacenar índices
            codes[:, m] = np.argmin(distances, axis=1)
        return codes

El cálculo de distancias aprovecha el *broadcasting* de los vectores de numpy y de `linalg.norm(...,axis=2)` para calcular la distancia rápidamente. Un equivalente con bucles `for` anidados es el siguiente:
```python
for i in range(N):
    for j in range(Ks):
        distances[i, j] = np.linalg.norm(sub_vectors[i] - centroids[j])
```
Se opta por usar numpy y *broadcasting* por su velocidad (usa C por debajo) y para enfocarnos en el algoritmo en si antes de sus detalles.

##### Complejidad

La complejidad está en el orden de $\mathcal{O}(NKD)$, ya que se calcula la distancia a $K$ vectores un total de $ND$ veces.

#### decode(codes)

Realiza el proceso inverso: dado un conjunto de índices, decodifica los centroides que representan según el *codebook*.

In [ ]:
    def decode(self, codes):
        N = codes.shape[0]
        X_reconstructed = np.zeros((N, self.M * self.Ds), dtype=np.float32)
        for m in range(self.M):
            centroids = self.codebooks[m]
            X_reconstructed[:, m*self.Ds:(m+1)*self.Ds] = centroids[codes[:, m]]
        return X_reconstructed


La decodificación se realiza ubicando los códigos del vector codificado en los *codebooks* generados para cada subvector. El vector reconstruido solo es una aproximación del vector original, puesto que se construye a partir de los vectores de los *codebooks*.

#### approximate_knn(query, codes, k)

La búsqueda de vecinos más cercanos no es exacta, puesto que se trabaja sobre la representación codificada del conjunto de vectores a sobre los que se trabaja. Sin embargo, la pérdida de presición es compensada por el ahorro sustancial de espacio en memoria, puesto que solo se tienen que cargar los *codebooks*, códigos y el vector de consulta, en lugar de todo el conjunto de vectores a analizar. De igual modo, se gana bastante velocidad, puesto que solo se calcula la distancia con los centroides en lugar de todo el conjunto de vectores.

Este método se basa en el cálculo asimétrico de distancia, puesto que se compara el vector de consulta con su precisión completa con los vectores codificados de menor precisión. Para esto, se calcula la distancia de los subvectores del vector de consulta con todos los centroides y se recuperan aquellas distancias cuyo código se encuentre en la tabla de códigos.

In [ ]:
    def approximate_knn(self, query, codes, k=5):
        N = codes.shape[0]
        distances = np.zeros(N, dtype=np.float32)

        for m in range(self.M):
            query_sub = query[m*self.Ds:(m+1)*self.Ds]
            centroids = self.codebooks[m]
            lookup = np.linalg.norm(centroids - query_sub, axis=1)
            distances += lookup[codes[:, m]]

        knn_indices = np.argsort(distances)[:k]
        knn_distances = distances[knn_indices]
        return knn_indices, knn_distances

##### Complejidad

La complejidad temporal llega a $\mathcal{O}(KD + NM)$, ya que se calcula la distancia a K centroides con dimensión total D y se buscan M distancias para N vectores. Esta complejidad es mucho menor que la de simple fuerza bruta que llega al orden $\mathcal{O}(ND)$.

## **Optimized Product Quantization**
En el Product Quantization tradicional, se divide cada vector en `M` subespacios fijos. Sin embargo, esta división puede ser arbitraria y no bóptima, las dimensiones más importantes pueden quedar separadas o mal alineadas, lo que aumenta el error de cuantización.

OPQ aplica una rotación lineal, al conjunto de vectores antes de dividirlos en subespacios. Esto reorienta los datos para que, una vez divididos, cada bloque contenga información más coherente internamente.


Aplicar una rotación en este contexto significa multiplicar los vectores por una matriz ortonormal `R`:
$$
x_{\text{rotado}} = x \cdot R
$$



Esto minimiza el error de reconstrucción al reordenar las direcciones importantes, mejora la precisión en búsquedas aproximadas, sin cambiar el contenido de los datos, solo su orientación.


### Diseño del algoritmo

Para implementar OPQ, se definió una clase OptimizedProductQuantizer que extiende la funcionalidad de un ProductQuantizer, añadiendo una etapa de rotación optimizada, cuenta con los métodos `fit(X)`, `encode(X)`, `decode(codes)` y `approximate_knn(query, codes)`.

In [ ]:
import numpy as np
from busqueda_aproximada.pq import ProductQuantizer

class OptimizedProductQuantizer:
    def __init__(self, M=4, Ks=256, num_iters=20):
        """
        M: número de subespacios
        Ks: número de centroides por subespacio
        num_iters: número de iteraciones para aprender la rotación óptima
        """
        self.M = M
        self.Ks = Ks
        self.num_iters = num_iters
        self.pq = ProductQuantizer(M, Ks)
        self.R = None

### fit(): Etapas del entrenamiento en OPQ
El objetivo de `fit` en OPQ es aprender una rotación óptima `R` que minimice el error de reconstrucción tras aplicar Product Quantization.


* La matriz de rotación R se inicializa como la identidad, lo que implica que inicialmente no se aplica ninguna transformación sobre los datos. En este punto, OPQ se comporta igual que un Product Quantizer tradicional.

$$
     R = I
$$
   
* Se aplica la rotación actual a todos los vectores del dataset:
$$
      X_{\text{rot}} = X \cdot R
$$


* Se entrena un Product Quantizer sobre los datos rotados, ajustando los centroides de cada subespacio mediante K-means..

* Se reconstruyen los vectores originales a partir de sus códigos utilizando los centroides obtenidos.

* Se optimiza la rotación R minimizando el error cuadrático entre los datos originales y los reconstruidos. Esto se logra usando la descomposición en valores singulares (SVD):
$$
      R = \text{argmin}_R \| X - (X_{\text{rot}} \cdot R^\top) \|^2
$$

* Este proceso se repite durante n iteraciones para refinar progresivamente la rotación y reducir el error de cuantización, el objetivo es minimizar el error entre los datos originales y los reconstruidos.



In [ ]:
def fit(self, X):
        """
        Aprende la matriz de rotación R y entrena los codebooks
        """
        N, D = X.shape
        self.R = np.eye(D, dtype=np.float32)

        for i in range(self.num_iters):
            # Paso 1: Rotar los datos
            X_rot = X @ self.R

            # Paso 2: Entrenar PQ sobre los datos rotados
            self.pq.fit(X_rot)

            # Paso 3: Reconstruir los datos desde los códigos
            X_codes = self.pq.encode(X_rot)
            X_reconstructed = self.pq.decode(X_codes)

            # Paso 4: Aprender nueva R usando SVD
            U, _, Vt = np.linalg.svd(X.T @ X_reconstructed)
            self.R = U @ Vt

        return self

### encode(X)
El método encode(X) tiene como objetivo mejorar la codificación que realiza un Product Quantizer tradicional. Para ello, antes de aplicar la cuantización, se rota el conjunto de vectores de entrada X mediante una matriz de rotación R previamente aprendida. Esta rotación reorienta los datos para que se adapten mejor a la partición de subespacios del PQ, lo que permite una representación más precisa con menor error de reconstrucción. Luego, sobre los vectores rotados, se aplica la codificación estándar de PQ, dividiendo cada vector en subespacios y asignando a cada uno su centroide más cercano.

In [ ]:
def encode(self, X):
        """
        Codifica los vectores aplicando la rotación y luego PQ.
        """
        X_rot = X @ self.R
        return self.pq.encode(X_rot)

### decode(codes)
El método decode(codes) reconstruye los vectores originales a partir de sus códigos cuantizados. Primero aplica la decodificación de PQ sobre los datos rotados y luego utiliza la transpuesta de la matriz de rotación R.T para invertir la rotación y devolver los vectores al espacio original.


In [ ]:
def decode(self, codes):
        """
        Reconstruye los vectores y aplica la rotación inversa.
        """
        X_rot_reconstructed = self.pq.decode(codes)
        return X_rot_reconstructed @ self.R.T

### approximate_knn(query, codes, k)
El método approximate_knn(query, codes, k) busca los k vecinos más cercanos a un vector de consulta. Para hacerlo, rota la consulta con la misma matriz R que se usó al codificar los datos, y luego realiza la búsqueda sobre el conjunto de códigos usando el método approximate_knn del Product Quantizer.

In [ ]:
def approximate_knn(self, query, codes, k=5):
        """
        Aplica approximate KNN a un vector de consulta rotado.
        """
        query_rot = query @ self.R
        return self.pq.approximate_knn(query_rot, codes, k=k)

## Análisis empírico de rendimiento

### Análisis de memoria

Por un lado el uso de memoria para LHS depende del tamaño del diccionario con el que se trabaja para almacenar los hashes de los vectores y sus buckets. Por otro lado, PQ y OPQ solo requieren almacenar un conjunto de índices enteros, los cuales aumentan linealmente con el número de datos introducidos (a lo más también se puede considerar el codebook, pero este aporta muy poco tamaño). Comparando ambas medidas en memoria, se ve que la indexación en LSH ocupa mucho más tamaño que PQ y OPQ. Esto demuestra la idea de que PQ y OPQ, como sus nombres indican, realizan una alta cuantización de los datos. En otras palabras, están optimizados para comprimir datos y almacenarlos en una fracción de su tamaño original.

Se puede ver que la indexación en PQ y OPQ aumentan lentamente, lo cual demuestra que la memoria requerida aumenta con el tamaño de los datos. Esto es de esperarse, puesto que se requieren almacenar más vectores de índices. De igual modo, no es sorprendente que PQ y OPQ ocupen el mismo espacio, puesto que solo cambia la forma en la que se almacenan los índices, no su cantidad. En el caso de LSH, este parece incrementar de forma lineal pero más pronunciada. Esto indicaría que en el diccionario se agregan tanto claves y valores al aumentar el número de datos a almacenar.

![Comparación de uso de memoria en indexación](https://raw.githubusercontent.com/aclj20/CC0E5-PC3/refs/heads/feature/profiling/plots/index_comparison.png)

### Análisis de tiempo

Mientras que PQ y OPQ demostraron su fortaleza en el almacenamiento en memoria, LSH demuestra ser superior en la velocidad de indexación. Este hecho tampoco es sorprendente: PQ y OPQ dependen de calcular distancia de todos los puntos a uno solo múltiples veces. Aunque la operación se realice entre vectores de menor dimensión, su impacto se ve en el tiempo considerable que toma para datos no tan grandes. OPQ sufre más al trabajar con matrices de rotaciones para cada vector, teniendo un aumento considerable al aumentar el número de datos. LSH, sin embargo, apenas sufre un aumento de costo temporal, puesto que sus operaciones son principalmente costantes y sin dependencia temporal fuerte al número de datos.

![Comparación de tiempo de indexación](https://raw.githubusercontent.com/aclj20/CC0E5-PC3/refs/heads/feature/profiling/plots/time_comparison.png)

### Análisis recall@k

Los análisis *recall@k* miden la precisión de la búsqueda de k vecinos comparando los k vecinos cercanos encontrados con los verdaderos k vecinos cercanos. Esto sirve para verificar la precisión en algoritmos de búsqueda aproximada de vecinos.

#### recall@5

Por un lado, el comportamiento de PQ y OPQ es el de esperarse. Ambos algoritmos tienen menos precisión encontrando vecinos cercanos cuando se aumentan el número de datos, puesto que los centroides van a agrupar cada vez vectores más dispersos o que codifiquen a los mismos centroides, lo cual resulta en búsquedas poco precisas. LSH, por otro lado, demuestra ser muy poco preciso para pocos datos, debido a la mayor cantidad de buckets vacíos en su estructura. Sin embargo, al aumentar la cantidad de datos, este es poco a poco más preciso hasta llegar a la medida recall de PQ y OPQ. Se espera que, al tener incluso más datos, supere a PQ y OPQ.

![Comparación de recall@5](https://raw.githubusercontent.com/aclj20/CC0E5-PC3/refs/heads/feature/profiling/plots/recall%405_comparison.png)

#### recall@10

Al aumentar el k hasta 10, el comportamiento se mantiene similar al de k = 5. PQ y OPQ tienen ligeramente mejor recall al retornar más candidatos, los cuales tienen mayor chance de ser mejores, mientras que LSH se reduce ligeramente por tener que buscar vecinos más lejanos en su tabla que pueden no ser tan precisos.

![Comparación de recall@10](https://raw.githubusercontent.com/aclj20/CC0E5-PC3/refs/heads/feature/profiling/plots/recall%4010_comparison.png)

## Limitaciones actuales y posibles mejoras

### Poco optimizado para grandes datos

Actualmente, el código desarrollado tiene poca tolerancia para grandes cantidades de datos. En particular, la búsqueda de vecinos cercanos fue bastante limitada, hasta 10000 datos, debido al tiempo excesivo que requerían estas pruebas. Esto se debe parcialmente al poco aprovechamiento de todo el CPU enel código desarrollado, enfocándose principalmente en demostrar el funcionamiento del algoritmo sobre su optimización para grandes datos. De igual modo, el uso de KMeans para PQ y OPQ no es lo suficientemente escalable y posiblemente sea un *bottleneck* para más de 10000 elementos. En LSH, se usa un diccionario de Python para almacenar los buckets y sus documentos almacenados, lo cual resulta demandante en su uso de memoria y limita su velocidad.

Para estos problemas, se pueden optar por librerías que aprovechen mejor los recursos de CPU o incluso de la GPU, como Faiss. De igual modo, para LSH, se pueden optar por estructuras alternativas a un diccionario de Python, como arreglos de NumPy.

### Inverted File Index - IVF

En PQ y OPQ, se puede implementar IVF para mejorar la velocidad de consulta y hacerlo más escalable. Este concepto se centra en buscar los vectores de consulta solo en regiones específicas en lugar de todos los vectores codificados, lo cual reduce el espacio de búsqueda y reduce el tiempo de consulta. Pese a no mejorar su medida recall, mejora su escalabilidad y tiempo de ejecución.

## Referencias
- https://towardsdatascience.com/similarity-search-product-quantization-b2a1a6397701/
- https://www.pinecone.io/learn/series/faiss/product-quantization/
- https://www.researchgate.net/publication/47815472_Product_Quantization_for_Nearest_Neighbor_Search
- https://softwaredoug.com/blog/2023/08/21/implementing-random-projections